### The purpose of this code is to identify the optimal commute division factor for a given proximity constraint

In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import os
from sklearn.cluster import DBSCAN, KMeans
from scipy.spatial.distance import pdist, squareform
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'geopy'

In [3]:
### only change values in this cell

DEFAULT_COMMUTE_RADIUS = 150 
MIN_SITES = 6

### Functions

In [4]:
def load_locations(file_path):
    """Load location data from Excel file"""
    return pd.read_excel(file_path)

def calculate_distance_miles(coord1, coord2):
    """Calculate distance between two coordinates in miles"""
    return geodesic(coord1, coord2).miles

def safe_join(series):
    """Join series values handling different data types and null values"""
    if series is None or len(series) == 0:
        return ""
    return ', '.join(str(x) for x in pd.Series(series).dropna().unique() if str(x).strip() != '')


# Campus Size Category based on quartiles of OPS_H sum
def get_size_category(ops_h_sum, q1, q2, q3):
    """Classify campus size based on OPS_H sum quartiles"""
    if ops_h_sum < q1:
        return "Small"
    elif ops_h_sum < q2:
        return "Medium"
    elif ops_h_sum < q3:
        return "Large"
    else:
        return "X-Large"


# HR Staffing Levels (base rules)
def get_hr_staffing_levels(size_category):
    """
    Returns recommended HR staffing levels based on size category.
    Base staffing:
      X-Large: Campus Leader=1, HR_L4=16, HR_L5=6, HR_L6=5, HR_L7=2
      Large:   Campus Leader=1, HR_L4=11, HR_L5=5, HR_L6=3, HR_L7=1
      Medium:  Campus Leader=1, HR_L4=8,  HR_L5=4, HR_L6=2, HR_L7=0
      Small:   Campus Leader=1, HR_L4=5,  HR_L5=3, HR_L6=2, HR_L7=0
      Multi-Site: Campus Leader=1, HR_L4=1, HR_L5=1, HR_L6=0, HR_L7=0
    """
    staffing_matrix = {
        'Small': {
            'Campus_Leader': 1,
            'HR_L4': 5,
            'HR_L5': 3,
            'HR_L6': 2,
            'HR_L7': 0
        },
        'Medium': {
            'Campus_Leader': 1,
            'HR_L4': 8,
            'HR_L5': 4,
            'HR_L6': 2,
            'HR_L7': 0
        },
        'Large': {
            'Campus_Leader': 1,
            'HR_L4': 11,
            'HR_L5': 5,
            'HR_L6': 3,
            'HR_L7': 1
        },
        'X-Large': {
            'Campus_Leader': 1,
            'HR_L4': 16,
            'HR_L5': 6,
            'HR_L6': 5,
            'HR_L7': 2
        },
        'Multi-Site': {
            'Campus_Leader': 1,
            'HR_L4': 1,
            'HR_L5': 1,
            'HR_L6': 0,
            'HR_L7': 0
        }
    }
    return staffing_matrix.get(size_category, {
        'Campus_Leader': 0,
        'HR_L4': 0,
        'HR_L5': 0,
        'HR_L6': 0,
        'HR_L7': 0
    })



# Analyze clusters and perform HR/staffing analysis.
def analyze_clusters(df):
    """Generate summary statistics and staffing analysis for each campus."""
    cluster_stats = []
    cluster_ids = df['cluster_id'].unique()
    
    # Compute quartile thresholds for OPS_H from valid (non-outlier) clusters.
    campus_sizes = []
    for cid in cluster_ids:
        cluster_data = df[df['cluster_id'] == cid]
        if not cluster_data['is_outlier'].all():
            campus_sizes.append(cluster_data['OPS_H'].sum())
    if campus_sizes:
        q1_size = np.percentile(campus_sizes, 25)
        q2_size = np.percentile(campus_sizes, 50)
        q3_size = np.percentile(campus_sizes, 75)
    else:
        q1_size, q2_size, q3_size = 15000, 30000, 50000

    for cid in cluster_ids:
        cluster_data = df[df['cluster_id'] == cid]
        center_lat = cluster_data['latitude'].mean()
        center_lon = cluster_data['longitude'].mean()
        
        max_distance = 0
        sites = cluster_data[['latitude', 'longitude']].values
        if len(sites) > 1:
            for i in range(len(sites)):
                for j in range(i+1, len(sites)):
                    d = calculate_distance_miles(sites[i], sites[j])
                    if d > max_distance:
                        max_distance = d
        
        cluster_aa_hc = cluster_data['OPS_H'].sum()
        cluster_ops_hc = cluster_data['OPS_S'].sum()
        
        # Current staffing summary (if such columns exist)
        current_staffing = {
            'Current_Campus_Leader': 1 if 'Campus_Leader' in cluster_data.columns else 0,
            'Current_HR_L4': cluster_data['HR_4'].sum() if 'HR_4' in cluster_data.columns else 0,
            'Current_HR_L5': cluster_data['HR_5'].sum() if 'HR_5' in cluster_data.columns else 0,
            'Current_HR_L6': cluster_data['HR_6'].sum() if 'HR_6' in cluster_data.columns else 0,
            'Current_HR_L7': cluster_data['HR_7'].sum() if 'HR_7' in cluster_data.columns else 0
        }
        current_hr_total = sum(current_staffing.values())
        current_gearing_ratio = (cluster_aa_hc / current_hr_total) if current_hr_total else None
        current_gearing_ratio_ops = (cluster_ops_hc / current_hr_total) if current_hr_total else None
        
        if cluster_data['multi_site'].any():
            size_category = "Multi-Site"
            new_hr_staffing = get_hr_staffing_levels("Multi-Site")
        else:
            size_category = get_size_category(cluster_aa_hc, q1_size, q2_size, q3_size)
            new_hr_staffing = get_hr_staffing_levels(size_category)
        
        new_hr_total = sum(new_hr_staffing.values())
        new_gearing_ratio = (cluster_aa_hc / new_hr_total) if new_hr_total else None
        new_gearing_ratio_ops = (cluster_ops_hc / new_hr_total) if new_hr_total else None
        
        stats = {
            'cluster_id': cid,
            'num_sites': len(cluster_data),
            'size_category': size_category,
            'Cluster_AA_HC': cluster_aa_hc,
            'Cluster_OPS_HC': cluster_ops_hc,
            'Current_Campus_Leader': current_staffing['Current_Campus_Leader'],
            'Current_HR_L4': current_staffing['Current_HR_L4'],
            'Current_HR_L5': current_staffing['Current_HR_L5'],
            'Current_HR_L6': current_staffing['Current_HR_L6'],
            'Current_HR_L7': current_staffing['Current_HR_L7'],
            'Current_HR_Total': current_hr_total,
            'Current_Gearing_Ratio_AA': current_gearing_ratio,
            'Current_Gearing_Ratio_OPS': current_gearing_ratio_ops,
            'New_Campus_Leader': new_hr_staffing['Campus_Leader'],
            'New_HR_L4': new_hr_staffing['HR_L4'],
            'New_HR_L5': new_hr_staffing['HR_L5'],
            'New_HR_L6': new_hr_staffing['HR_L6'],
            'New_HR_L7': new_hr_staffing['HR_L7'],
            'New_HR_Total': new_hr_total,
            'New_Gearing_Ratio_AA': new_gearing_ratio,
            'New_Gearing_Ratio_OPS': new_gearing_ratio_ops,
            'country': safe_join(cluster_data['country']),
            'markets': safe_join(cluster_data['market']),
            'cities': safe_join(cluster_data['city']),
            'states': safe_join(cluster_data['state']),
            'center_latitude': center_lat,
            'center_longitude': center_lon,
            'max_distance_miles': round(max_distance, 2),
            'num_reassigned_outliers': len(cluster_data[cluster_data['reassignment_type'].notnull()])
        }
        cluster_stats.append(stats)
    
    df_stats = pd.DataFrame(cluster_stats)
    # Round gearing ratios for presentation.
    for col in ['Current_Gearing_Ratio_AA', 'Current_Gearing_Ratio_OPS',
                'New_Gearing_Ratio_AA', 'New_Gearing_Ratio_OPS']:
        df_stats[col] = df_stats[col].apply(lambda x: round(x, 2) if pd.notnull(x) else x)
    
    return df_stats

### Main

In [ ]:
##change path for input file
df = pd.read_excel('locations.xlsx')

## distance matrix b/w site locations

coords = df[['latitude', 'longitude']].values
dist_matrix = squareform(pdist(coords, metric=lambda u, v: calculate_distance_miles(u, v)))

### clustering 1st pass
df_clustered = df.copy()

min_sites = MIN_SITES

commute_division_options = [1,2,3,4,5,6,7,8,9,10]
i=0

num_clusters_list = []
num_outliers_list = []
perct_outliers_list = []
num_clusters_above_commute_list = []
perct_clusters_above_commute_list = []

while i<len(commute_division_options):
    print(i)
    commute_radius = DEFAULT_COMMUTE_RADIUS/commute_division_options[i]
    clustering = DBSCAN(eps=commute_radius, min_samples=min_sites, metric='precomputed')
    cluster_labels = clustering.fit_predict(dist_matrix)
    df_clustered['cluster_id'] = cluster_labels
    df_clustered['initial_outlier'] = (cluster_labels == -1)
    df_clustered['outlier_reassigned'] = False
    df_clustered['is_outlier'] = (cluster_labels == -1)
    df_clustered['multi_site'] = False
    df_clustered['reassignment_type'] = None

    #number of clusters
    num_clusters_list.append(df_clustered['cluster_id'].nunique())

    ### Analyse initial cluster
    cluster_analysis = analyze_clusters(df_clustered)

    ### #outliers grouped into -1 cluster
    num_outliers_list.append(cluster_analysis[(cluster_analysis['cluster_id'] == -1)]['num_sites'].sum())
    perct_outliers_list.append(cluster_analysis[(cluster_analysis['cluster_id'] == -1)]['num_sites'].sum() / cluster_analysis['num_sites'].sum())

    ## except outliers, summary of remaining clusters
    cluster_analysis_no_outlier = cluster_analysis[~(cluster_analysis['cluster_id'] == -1)]

    ##how many clusters have max_distance > DEFAULT_COMMUTE_RADIUS
    num_clusters_above_commute_list.append(len(cluster_analysis_no_outlier[cluster_analysis_no_outlier['max_distance_miles'] > DEFAULT_COMMUTE_RADIUS]))
    perct_clusters_above_commute_list.append(len(cluster_analysis_no_outlier[cluster_analysis_no_outlier['max_distance_miles'] > DEFAULT_COMMUTE_RADIUS])/ len(cluster_analysis_no_outlier))

    ##storing cluster data
    #cluster_data.append(dict(zip(num_clusters,num_outliers.tolist(),perct_outliers.tolist(),num_clusters_above_commute,perct_clusters_above_commute)))
    #cluster_summary = cluster_analysis_no_outlier[['num_sites','Cluster_AA_HC','max_distance_miles']].describe()

    i+=1

0
1
2
3
4
5
6
7
8
9


In [6]:
report = pd.DataFrame(list(zip(
                commute_division_options,
                num_clusters_list,
                num_outliers_list,
                perct_outliers_list,
                num_clusters_above_commute_list,
                perct_clusters_above_commute_list,
            )),
        columns=["commute_division_options","num_clusters","num_outliers","perct_outliers","num_clusters_with_max_dist_above_commute","perct_clusters_with_max_dist_above_commute",])
report

,commute_division_options,num_clusters,num_outliers,perct_outliers,num_clusters_with_max_dist_above_commute,perct_clusters_with_max_dist_above_commute
0,1,9,21,0.018784,7,0.875000
1,2,31,109,0.097496,11,0.366667
2,3,36,215,0.192308,5,0.142857
3,4,38,251,0.224508,3,0.081081
4,5,40,274,0.245081,2,0.051282
5,6,43,301,0.269231,1,0.023810
6,7,46,321,0.287120,1,0.022222
7,8,51,350,0.313059,1,0.020000
8,9,52,397,0.355098,1,0.019608
9,10,51,458,0.409660,1,0.020000


In [31]:
### best commute division option

min_clusters_with_max_dist_above_commute = int(report['num_clusters_with_max_dist_above_commute'].min())

best_option = report[report['num_clusters_with_max_dist_above_commute']==min_clusters_with_max_dist_above_commute]

min_outliers = int(best_option['num_outliers'].min())

best_option = report[report['num_outliers']==min_outliers]

### best commute divison option for given scenario
print(int(best_option['commute_division_options']))


6


/tmp/ipykernel_9813/2923228575.py:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(int(best_option['commute_division_options']))
